In [11]:

!pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib


Defaulting to user installation because normal site-packages is not writeable
  Using cached google_api_python_client-2.159.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_api_core-2.24.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached proto_plus-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
Using cached google_api_python_clie

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

def authenticate_gmail():
    # Path to the credentials.json file
    creds_path = "credentials.json"

    # Authenticate with the Gmail API
    flow = InstalledAppFlow.from_client_secrets_file(creds_path, scopes=['https://www.googleapis.com/auth/gmail.modify'])
    creds = flow.run_local_server(port=0)
    return build('gmail', 'v1', credentials=creds)

# Example usage
if __name__ == "__main__":
    service = authenticate_gmail()
    # Your Gmail API code here


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=312350626782-hr5mvakbs32693qi88b0pi6pqs6q1pcr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50658%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=a46Lwuc5ams4IBa7hEqgYeatYakW3I&access_type=offline


In [13]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    """
    Authenticate Gmail API using the provided credentials.json file.
    """
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)
    return service


In [ ]:
import os
import base64
import time
import redis
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from apscheduler.schedulers.background import BackgroundScheduler

# Setting up Redis for queuing emails
email_queue = redis.Redis(host='localhost', port=6379, db=0)

# Gmail API Authentication
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token_file:
            token_file.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

# Function to list and handle emails
def process_emails(service):
    try:
        results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread").execute()
        messages = results.get('messages', [])

        if not messages:
            print("No new emails.")
            return

        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            thread_id = msg['threadId']
            sender = get_header_value(msg, 'From')
            subject = get_header_value(msg, 'Subject')

            print(f"Found email from: {sender} with subject: {subject}")

            # Simulate a response and add to queue
            reply_text = f"Thanks for your email. We will get back to you soon!"
            email_queue.lpush("email_queue", {"email_id": message['id'], "reply_text": reply_text})

    except HttpError as error:
        print(f"An error occurred: {error}")

# Function to reply to emails from queue
def reply_from_queue(service):
    while email_queue.llen("email_queue") > 0:
        email = email_queue.rpop("email_queue")
        email_data = eval(email)  # Convert string back to dictionary
        reply_to_email(service, email_data["email_id"], email_data["reply_text"])

# Function to send reply
def reply_to_email(service, email_id, reply_text):
    try:
        message = service.users().messages().get(userId='me', id=email_id).execute()
        thread_id = message['threadId']
        subject = get_header_value(message, 'Subject')
        sender = get_header_value(message, 'From')

        reply_message = f"To: {sender}\nSubject: Re: {subject}\nContent-Type: text/plain; charset='UTF-8'\n\n{reply_text}"
        raw_message = {'raw': base64.urlsafe_b64encode(reply_message.encode('UTF-8')).decode('ascii')}
        service.users().messages().send(userId='me', body=raw_message, threadId=thread_id).execute()
        print(f"Replied to email with subject: {subject}")
    except HttpError as error:
        print(f"An error occurred: {error}")

# Helper function to get header values
def get_header_value(message, header_name):
    headers = message.get('payload', {}).get('headers', [])
    for header in headers:
        if header['name'] == header_name:
            return header['value']
    return "N/A"

# Setup email processing with scheduler
def main():
    service = authenticate_gmail()
    scheduler = BackgroundScheduler()
    scheduler.add_job(lambda: process_emails(service), 'interval', minutes=5)  # Check for new emails every 5 minutes
    scheduler.add_job(lambda: reply_from_queue(service), 'interval', minutes=10)  # Reply to emails every 10 minutes
    scheduler.start()

    try:
        while True:
            time.sleep(1)
    except (KeyboardInterrupt, SystemExit):
        scheduler.shutdown()

if __name__ == "__main__":
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=312350626782-hr5mvakbs32693qi88b0pi6pqs6q1pcr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51464%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=KiD871SHb394o6VhjhuIvkTLp5sh97&access_type=offline


Run time of job "main.<locals>.<lambda> (trigger: interval[0:05:00], next run at: 2025-01-25 13:25:49 IST)" was missed by 0:02:45.221224
Run time of job "main.<locals>.<lambda> (trigger: interval[0:10:00], next run at: 2025-01-26 00:10:49 IST)" was missed by 0:02:45.240588
Job "main.<locals>.<lambda> (trigger: interval[0:05:00], next run at: 2025-01-26 00:10:49 IST)" raised an exception
Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Roaming\Python\Python311\site-packages\httplib2\__init__.py", line 1366, in _conn_request
    conn.connect()
  File "C:\Users\lenovo\AppData\Roaming\Python\Python311\site-packages\httplib2\__init__.py", line 1142, in connect
    address_info = socket.getaddrinfo(host, port, 0, socket.SOCK_STREAM)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^

An error occurred: <HttpError 403 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages?labelIds=INBOX&q=is%3Aunread&alt=json returned "Gmail API has not been used in project 312350626782 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/gmail.googleapis.com/overview?project=312350626782 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'message': 'Gmail API has not been used in project 312350626782 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/gmail.googleapis.com/overview?project=312350626782 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.', 'domain': 'usageLimits', 'reason': 'accessNotConfigured', 'extendedHelp': 'https://console.developers.google.com'}]">


In [18]:
!pip install redis


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/261.5 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/261.5 kB 262.6 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/261.5 kB 469.7 kB/s eta 0:00:01
   ------------------------- ------------ 174.1/261.5 kB 952.6 kB/s eta 0:00:01
   ---------------------------------------- 261.5/261.5 kB 1.3 MB/s eta 0:00:00


In [20]:
!pip install apscheduler


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/64.0 kB ? eta -:--:--
   ------ --------------------------------- 10.2/64.0 kB ? eta -:--:--
   ------------------- -------------------- 30.7/64.0 kB 435.7 kB/s eta 0:00:01
   -------------------------------------- - 61.4/64.0 kB 656.4 kB/s eta 0:00:01
   ---------------------------------------- 64.0/64.0 kB 493.2 kB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.


In [2]:
!cd C:\Users\lenovo\Desktop\Email_proj\App\Email_auto.ipynb

The directory name is invalid.
